In [24]:
import pandas
import glob
import numpy

In [81]:
files = glob.glob("*.csv")

for x in files:
   if (x[0] == "C"):
    pass
   else:
       print(x)
       df = pandas.read_csv(x)
        
        # Remove all put and call holdings for now
       df= df[df.putCall != 'Call']
       df= df[df.putCall != 'Put']
        
        # Remove the PRN holdings
       df= df[df.sshPrnamtType != 'PRN']
        
        #Combine all duplicate entries
       df = df.groupby(['nameOfIssuer']).sum()
     
        #Remove columns that we don't need
       df = df[df.columns.difference(['Sole', 'Shared','None','titleOfClass','cusip','sshPrnamtType','investmentDiscretion','otherManager','putCall'])]
       
          #
       df['sprice'] = (df['value'] / df['sshPrnamt']) *1000 
       df = df[df.columns.difference(['value'])]
        
       df.to_csv("Clean"+x)
        
      
pandas.read_csv("Clean0214.csv")

0214.csv
0514.csv
0814.csv
1114.csv


,nameOfIssuer,sprice,sshPrnamt
0,1 800 FLOWERS COM,5.409870,48430
1,1ST SOURCE CORP,31.928269,60448
2,3-D SYS CORP DEL,92.933651,158059
3,3M CO,140.249912,1904593
4,51JOB INC,77.578051,1057
5,58 COM INC,38.362069,16240
6,8X8 INC NEW,10.130041,21224
7,AAON INC,31.952130,181991
8,AAR CORP,28.008015,307412
9,AARONS INC,29.397647,145828


In [87]:
#retrieve and sort the cleaned files
cfiles = glob.glob("*.csv")
cfiles = sorted(cfiles)

#make our final dataframe to populate
fdf = pandas.DataFrame()


merged = pandas.read_csv(cfiles[4])
for x in range(int(len(cfiles)/2)+1,len(cfiles)):
    merged = pandas.merge(merged, pandas.read_csv(cfiles[x]), on = ['nameOfIssuer','nameOfIssuer'])

#accumulator for column names
colnames = ['Company']

    
for x in range(0,len(cfiles)):
    if ((cfiles[x])[0] == "C"):
        
        #Making the column headers for our new table from our old ones
        amt = (cfiles[x])[5:9] + "price"
        mon = (cfiles[x])[5:9] + "amt"    
        colnames.append(amt)
        colnames.append(mon)
        
        
        #add columns for calculated difference in number of shares
        if (x < len(cfiles)-1):
        
            first = (cfiles[x])[5:9]
            second = (cfiles[x+1])[5:9]
            diff= first + "-" + second + "chng"
            colnames.append(diff)
        
            merged[diff]= merged[(second+"amt")] - merged[(first+"amt")]
        
        
        
        
merged.columns = colnames
merged

KeyError: '0514amt'